In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms

import time
import copy

from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [19]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
    
print(DEVICE)

cuda


In [20]:
BATCH_SIZE = 256
EPOCHS = 20

In [21]:
# 이미지 크기 조정 (64,64) , 이미지를 텐서 형태로 변환하고 0~1로 정규화
transform_base = transforms.Compose([transforms.Resize((64,64)),transforms.ToTensor()])

train_dataset = ImageFolder(root='./splited/train/', transform = transform_base)
val_dataset = ImageFolder(root='./splited/val/', transform = transform_base)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = BATCH_SIZE, shuffle = True, num_workers=4)

#train_loader 데이터 형식 (data,target)

In [22]:
# convolution 연산 3->16->32->64

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        
        self.conv1 = torch.nn.Conv2d(3, 16, 3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, 3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(32, 64, 3, stride=1, padding=1)
        
        self.pool = torch.nn.MaxPool2d(2,2)
        
        self.fc1 = torch.nn.Linear(4096, 512)  # conv 연산을 3번 진행 하면서 pool링 과정을 3번 거치면 64,64 -> 8,8로 축소된다.\
        self.fc2 = torch.nn.Linear(512, 128)
        self.fc3 = torch.nn.Linear(128, 33)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25,training=self.training)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25,training=self.training)
        
        x = self.conv3(x)
        x = F.relu(x)
        x = self.pool(x)
        x = F.dropout(x, p=0.25,training=self.training)
        
        x = x.view(-1, 4096)  
        
        x = self.fc1(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc2(x)
        x = F.relu(x)
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.fc3(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [23]:
model_cnn = CNN().to(DEVICE)
optimizer = optim.Adam(model_cnn.parameters(), lr=0.001)

In [24]:
def train(model, train_loader, optimizer):
    model.train()
    
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        
        output = model(data)
        
        loss = F.cross_entropy(output, target)
        
        loss.backward()
        optimizer.step()

In [25]:
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss += F.cross_entropy(output, target, reduction='sum').item()
            
            pred = output.max(1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    return test_loss, test_accuracy

In [26]:
def train_baseline(model ,train_loader, val_loader, optimizer, num_epochs = 30):
    best_acc = 0.0  
    best_model_wts = copy.deepcopy(model.state_dict()) 
 
    for epoch in range(1, num_epochs + 1):
        since = time.time()  
        train(model, train_loader, optimizer)
        train_loss, train_acc = evaluate(model, train_loader) 
        val_loss, val_acc = evaluate(model, val_loader)
        
        if val_acc > best_acc: 
            best_acc = val_acc 
            best_model_wts = copy.deepcopy(model.state_dict())
        
        time_elapsed = time.time() - since 
        print('-------------- epoch {} ----------------'.format(epoch))
        print('train Loss: {:.4f}, Accuracy: {:.2f}%'.format(train_loss, train_acc))   
        print('val Loss: {:.4f}, Accuracy: {:.2f}%'.format(val_loss, val_acc))
        print('Completed in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) 
    model.load_state_dict(best_model_wts)  
    return model
 

base = train_baseline(model_cnn, train_loader, val_loader, optimizer, EPOCHS)  	 #(16)
torch.save(base,'baseline.pt')

-------------- epoch 1 ----------------
train Loss: 2.3379, Accuracy: 36.48%
val Loss: 2.3343, Accuracy: 36.89%
Completed in 1m 6s
-------------- epoch 2 ----------------
train Loss: 1.5925, Accuracy: 54.93%
val Loss: 1.6020, Accuracy: 55.08%
Completed in 1m 6s
-------------- epoch 3 ----------------
train Loss: 1.2725, Accuracy: 63.86%
val Loss: 1.2964, Accuracy: 63.47%
Completed in 1m 7s
-------------- epoch 4 ----------------
train Loss: 1.0525, Accuracy: 68.76%
val Loss: 1.0847, Accuracy: 67.68%
Completed in 1m 7s
-------------- epoch 5 ----------------
train Loss: 0.8612, Accuracy: 74.11%
val Loss: 0.8965, Accuracy: 73.25%
Completed in 1m 6s
-------------- epoch 6 ----------------
train Loss: 0.7431, Accuracy: 77.59%
val Loss: 0.7899, Accuracy: 76.09%
Completed in 1m 6s
-------------- epoch 7 ----------------
train Loss: 0.6522, Accuracy: 79.71%
val Loss: 0.7031, Accuracy: 78.57%
Completed in 1m 6s
-------------- epoch 8 ----------------
train Loss: 0.5927, Accuracy: 82.03%
val Lo